In [1]:
from policy import PIB, NEATProperty, NEATPolicy, NEATRequest
from cib import CIB

# Application Request

An application would like to open a new TCP connection using NEAT to a destination host `d1` with the IP 10.1.23.45. Further the MTU should be 1500 bytes if possible. We can configure this NEATRequest as follows: 

In [2]:
property1 = NEATProperty(('remote_ip', '10.1.23.45'), level=NEATProperty.IMMUTABLE)

request = NEATRequest()
request.properties.insert(property1)
request.properties.insert(NEATProperty(('MTU', (1500, float('inf')))))
request.properties.insert(NEATProperty(('transport', 'TCP')))

print(request)
request.properties

<NEATRequest: 0 candidates, 3 properties>


{[remote_ip|10.1.23.45], (MTU|1500-inf), (transport|TCP)}

# Example Scenario

Consider a host with three local interfaces `en0`, `en1`, `ra0`. Two of the interfaces, `en0`, `en1`, are wired while `ra0` is a 3G interface.

In [3]:
cib = CIB('cib/example/')

loading CIB source A.connection
loading CIB source B.connection
loading CIB source C.connection


The currently known network properties are stored in the following CIB entries:

In [4]:
cib.dump()

C: {[MTU|890], [is_wired|False], [remote_ip|10.1.23.45], [interface|ra0], [capacity|0.6], [local_ip|10.10.2.2]}
B: {[MTU|1500], [is_wired|True], <transport_UDP|true>, [interface|en1], <transport_TCP|true>, [capacity|40], [local_ip|192.168.1.2]}
A: {[MTU|9600], <dns_name|backup.example.com>, [is_wired|True], <transport|TCP>, [remote_ip|10.1.23.45], [capacity|10], [interface|en0], [local_ip|10.2.0.1]}


# PIB 
We define two policies and add them to the Policy Information Base (PIB).

A "bulk transfer" policy is configured on the host. This policy is triggered by a specific destination IP, which is known to be the address of backup NFS share:

In [5]:
policy1 = NEATPolicy(name='Bulk transfer')
policy1.match.insert(NEATProperty(('remote_ip', '10.1.23.45')))
policy1.properties.insert(NEATProperty(('capacity', (10, 100)), level=NEATProperty.IMMUTABLE))
policy1.properties.insert(NEATProperty(('MTU', 9600)))

Another policy is in place to enable TCP window scaling on 10G links, if possible:

In [6]:
policy2 = NEATPolicy(name='TCP options')
policy2.match.insert(NEATProperty(('MTU', 9600)))
policy2.match.insert(NEATProperty(('is_wired', True)))
policy2.properties.insert(NEATProperty(('TCP_window_scale', True)))

In [7]:
pib = PIB()
pib.register(policy2)
pib.register(policy1)
pib.dump()

===== PIB START =====
POLICY Bulk transfer: {(remote_ip|10.1.23.45)}  ==>  {(MTU|9600), [capacity|10-100]}
POLICY TCP options: {(MTU|9600), (is_wired|True)}  ==>  {(TCP_window_scale|True)}
===== PIB END =====


# Lookup Result

## CIB Lookup

First we perform a lookup in the CIB. The NEAT request yields three candidates:


In [8]:
cib.lookup(request)
request.dump()

{[remote_ip|10.1.23.45], (MTU|1500-inf), (transport|TCP)}
===== candidates =====
[0]PROPERTIES: {[MTU|890]-1.0, [is_wired|False], (transport|TCP), [remote_ip|10.1.23.45]+1.0, [local_ip|10.10.2.2], [capacity|0.6], [interface|ra0]}, POLICIES: set()
[1]PROPERTIES: {[MTU|1500]+1.0, [is_wired|True], <transport_UDP|true>, (transport|TCP), [remote_ip|10.1.23.45], <transport_TCP|true>, [capacity|40], [interface|en1], [local_ip|192.168.1.2]}, POLICIES: set()
[2]PROPERTIES: {[MTU|9600]+1.0, [is_wired|True], (transport|TCP)+1.0, [remote_ip|10.1.23.45]+1.0, [local_ip|10.2.0.1], [capacity|10], <dns_name|backup.example.com>, [interface|en0]}, POLICIES: set()
===== candidates =====


The scores of the CIB properties that match the request properties have been increased

## PIB Lookup
In the next step the policies are applied starting with the "Bulk transfer" policy which has the smallest number of match entries.


In [9]:
pib.lookup_all(request.candidates)
request.dump()

Candidate 0 is invalidated due to policy
{[remote_ip|10.1.23.45], (MTU|1500-inf), (transport|TCP)}
===== candidates =====
[0]PROPERTIES: {[MTU|1500]+0.0, [is_wired|True], (TCP_window_scale|True), <transport_UDP|true>, (transport|TCP), [remote_ip|10.1.23.45], <transport_TCP|true>, [capacity|40]+1.0, [interface|en1], [local_ip|192.168.1.2]}, POLICIES: {4381688384, 4381687936}
[1]PROPERTIES: {[MTU|9600]+2.0, [is_wired|True], (TCP_window_scale|True), (transport|TCP)+1.0, [remote_ip|10.1.23.45]+1.0, [local_ip|10.2.0.1], [capacity|10]+1.0, <dns_name|backup.example.com>, [interface|en0]}, POLICIES: {4381688384, 4381687936}
===== candidates =====


Candidate 1 becomes:        

In [10]:
request.candidates[0].dump()

PROPERTIES: {[MTU|1500]+0.0, [is_wired|True], (TCP_window_scale|True), <transport_UDP|true>, (transport|TCP), [remote_ip|10.1.23.45], <transport_TCP|true>, [capacity|40]+1.0, [interface|en1], [local_ip|192.168.1.2]}, POLICIES: {4381688384, 4381687936}


Next we examine Candidate 2:

In [11]:
request.candidates[1].dump()

PROPERTIES: {[MTU|9600]+2.0, [is_wired|True], (TCP_window_scale|True), (transport|TCP)+1.0, [remote_ip|10.1.23.45]+1.0, [local_ip|10.2.0.1], [capacity|10]+1.0, <dns_name|backup.example.com>, [interface|en0]}, POLICIES: {4381688384, 4381687936}


Note that the score of the MTU property was reduced, as it did not match the requested property of the "Bulk transfer" policy.

The "TCP options" policy is not applied as the candidate does not match the policy's MTU property.

---

The third candidate was invalidated because the "Bulk transfer" policy contains an immutable property requiring a capacity of 10G, which candidate 3 cannot fulfil.

---

# NEAT Logic
The two candidates can now be passed on to the NEAT logic as JSON strings

In [12]:
request.candidates[0].properties.json()

'[{"MTU": {"level": 2, "score": 0.0, "value": 1500}}, {"is_wired": {"level": 2, "score": NaN, "value": true}}, {"TCP_window_scale": {"level": 1, "score": NaN, "value": true}}, {"transport_UDP": {"level": 0, "score": NaN, "value": "true"}}, {"transport": {"level": 1, "score": NaN, "value": "TCP"}}, {"remote_ip": {"level": 2, "score": NaN, "value": "10.1.23.45"}}, {"transport_TCP": {"level": 0, "score": NaN, "value": "true"}}, {"capacity": {"level": 2, "score": 1.0, "value": 40}}, {"interface": {"level": 2, "score": NaN, "value": "en1"}}, {"local_ip": {"level": 2, "score": NaN, "value": "192.168.1.2"}}]'

In [13]:
request.candidates[1].properties.json()

'[{"MTU": {"level": 2, "score": 2.0, "value": 9600}}, {"is_wired": {"level": 2, "score": NaN, "value": true}}, {"TCP_window_scale": {"level": 1, "score": NaN, "value": true}}, {"transport": {"level": 1, "score": 1.0, "value": "TCP"}}, {"remote_ip": {"level": 2, "score": 1.0, "value": "10.1.23.45"}}, {"local_ip": {"level": 2, "score": NaN, "value": "10.2.0.1"}}, {"capacity": {"level": 2, "score": 1.0, "value": 10}}, {"dns_name": {"level": 0, "score": NaN, "value": "backup.example.com"}}, {"interface": {"level": 2, "score": NaN, "value": "en0"}}]'